# Confusion Matrix

In [ ]:

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
        https://stackoverflow.com/a/50386871
    """

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

# Data Pickling

In [ ]:
#Import statements
import pickle
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, roc_curve

In [ ]:
with open('../../data/processed/full_set.pkl', 'rb') as f:
    full_set = pickle.load(f)
with open('../../data/processed/train_set.pkl', 'rb') as f:
    train_set = pickle.load(f)
with open('../../data/processed/test_set.pkl', 'rb') as f:
    test_set = pickle.load(f)
with open('../../data/processed/train_set_30.pkl', 'rb') as f:
    train_set_30 = pickle.load(f)
with open('../../data/processed/test_set_30.pkl', 'rb') as f:
    test_set_30 = pickle.load(f)
with open('../../data/processed/train_set_70.pkl', 'rb') as f:
    train_set_70 = pickle.load(f)
with open('../../data/processed/test_set_70.pkl', 'rb') as f:
    test_set_70 = pickle.load(f)

In [ ]:
test_set_70.head()
train_set.head()

In [ ]:
# Split into X, y format 
X_train = train_set.T.iloc[:-1].T
y_train = train_set.T.iloc[-1].T

X_test = test_set.T.iloc[:-1].T
y_test = test_set.T.iloc[-1].T

In [ ]:
y_train.head()

# Multilayer Perceptron

### Experiment with various Neural Network parameters: add or remove nodes, layers and connections, vary the learning rate, epochs and momentum, and validation threshold.

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
def get_mlp():
    mlp = MLPClassifier(random_state=42
                        # more non default parameters? E.g. only stochastic gradient descent has been covered in lectures 
                        # also doesn't reach convergence before timeout with current settings
                       )
    return mlp

### Trying it using keras:

In [ ]:
from numpy.random import seed
import random
import tensorflow as tf
np.random.seed(42)
random.seed(42)
from tensorflow import keras
# tf.random.set_random_seed(42)
tf.random.set_seed(42)

from keras.wrappers.scikit_learn import KerasClassifier

# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# https://machinelearningmastery.com/build-multi-layer-perceptron-neural-network-models-keras/
from keras.models import Sequential
from keras.layers import Dense

def build_keras_model():
    model = Sequential()
    model.add(Dense(1024, input_dim=2304, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    print(model.summary())

    # compile the keras model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fit the keras model on the dataset
# model.fit(X_train, y_train, epochs=10, batch_size=100)

# evaluate the keras model
# loss, accuracy = model.evaluate(X_test, y_test)

kcls = KerasClassifier(build_fn=build_keras_model, epochs=10, batch_size=100)

#Note, if you try running this example in an IPython or Jupyter notebook you may get an error.
#The reason is the output progress bars during training. 
#You can easily turn these off by setting verbose=0 in fit() and evaluate() calls



In [ ]:
# https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/
from sklearn.model_selection import StratifiedKFold
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=42)
# todo run this
def cross_validate(classifier, X, y):
    '''    
    Given a classifier and training data:
        * Do 10fold CV
        * average the scores
    What this means is for the caller to interpret.
    Returns average result over CV runs 
    '''
    standardising_classifier = make_pipeline(preprocessing.StandardScaler(), classifier)
    scores = cross_val_score(standardising_classifier, X, y, cv=10)
    return np.mean(scores)

# Running An Experiment

In [ ]:
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
import pandas as pd

from sklearn.metrics import roc_auc_score

def run_experiment(classifier, X_train_data, y_train_data, X_test_data, y_test_data):
    '''
    Given a classifier, training data, and test data:
    * Train on the training data
    * Test on the test data
    * display the confusion matrix and other metrics
    '''
    classifier_pipe = make_pipeline(preprocessing.StandardScaler(), classifier)

    classifier_pipe.fit(X_train_data, y_train_data)
    y_pred = classifier_pipe.predict(X_test_data)

    conf_mat = confusion_matrix(y_test_data, y_pred)
    print(classification_report(y_test_data, y_pred))
    plot_confusion_matrix(conf_mat, target_names=y_test_data.unique().sort())
    
    #Conf matrix quantities to give an idea of TP/FP/TN/FN
    y_test_data_df = pd.DataFrame(y_test_data)
    y_pred_data_df = pd.DataFrame(y_pred)
    cnf_matrix = confusion_matrix(y_test_data_df.values, y_pred_data_df.values)
    
    # https://stackoverflow.com/questions/50666091/true-positive-rate-and-false-positive-rate-tpr-fpr-for-multi-class-data-in-py
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    
    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN)

    print(f"True Positive Rate: {TPR}\nFalse Positive Rate: {FPR}\n")
    
    #AUC
#     print(roc_auc_score(y_test_data, y_pred))

    

## Linear Classifier

In [ ]:
from sklearn.svm import SVC
linclf = SVC(kernel="linear")
print("Linear Classifier, original split:")
print(f"{train_set['target'].shape[0]} training examples")
run_experiment(linclf, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")

## Neural Nets

In [ ]:
mlp_max = get_mlp()
cv = cross_validate(mlp_max, full_set.iloc[:,:-1], full_set['target'])
print(cv)
# run_experiment(mlp_max, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])

# 5-7: Using different sized train/test sets instead of cross validation

In [ ]:
print("Original split:")
print(f"{train_set['target'].shape[0]} training examples")
mlp_max = get_mlp()
run_experiment(mlp_max, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")


print("Reduced training set:")
print(f"{train_set_30['target'].shape[0]} training examples")
mlp_mid = get_mlp()
run_experiment(mlp_mid, train_set_30.iloc[:,:-1], train_set_30['target'], test_set_30.iloc[:,:-1], test_set_30['target'])
print("------\n")


print("Smallest training set")
print(f"{train_set_70['target'].shape[0]} training examples")
mlp_small = get_mlp()
run_experiment(mlp_small, train_set_70.iloc[:,:-1], train_set_70['target'], test_set_70.iloc[:,:-1], test_set_70['target'])
print("------\n")


... So the results of the previous cell are a bit too good. Let's keep going with the training set reduction:

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# half the remaining training data
def half_remaining_training_data(train_X, train_y, test_X, test_y):
    X_train_tiny, X_test_tiny_tmp, y_train_tiny, y_test_tiny_tmp = train_test_split(train_X, train_y, test_size=0.5, random_state=42)
    
    # Move the new test split into the existing test data
    X_test_tiny = pd.concat([X_test_tiny_tmp, test_X], axis=0)
    y_test_tiny = pd.concat([y_test_tiny_tmp, test_y], axis=0)
                             
    return X_train_tiny, X_test_tiny, y_train_tiny, y_test_tiny



In [ ]:
X_train_90, X_test_90, y_train_90, y_test_90 = half_remaining_training_data(train_set_70.iloc[:,:-1], train_set_70['target'], test_set_70.iloc[:,:-1], test_set_70['target'])
X_train_95, X_test_95, y_train_95, y_test_95 = half_remaining_training_data(X_train_90, y_train_90, X_test_90, y_test_90)

In [ ]:
print("10% training set")
print(f"{y_train_90.shape[0]} training examples")
mlp_tiny = get_mlp()
run_experiment(mlp_tiny, X_train_90, y_train_90, X_test_90, y_test_90)

print("5% training set")
print(f"{y_train_95.shape[0]} training examples")
mlp_tinier = get_mlp()
run_experiment(mlp_tinier, X_train_95, y_train_95, X_test_95, y_test_95)


In [ ]:
print("Keras model, original data split:")
print(f"{train_set['target'].shape[0]} training examples")

run_experiment(kcls, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")

## Experiments with CNNs

In [ ]:
# Experiments in data reshaping to check techniques to be used in the CNN architecture
print(train_set.shape)
print(X_train_95.shape)
X_tmp = train_set.iloc[:,:-1]
print(X_tmp.shape)

reshaped_train = X_train_95.to_numpy().reshape(-1, 48, 48)
print("y_train_95.shape: ", y_train_95.shape)

plt.figure()
plt.imshow(reshaped_train[0], cmap='Greys')
plt.title(y_train_95.iloc[0])
plt.show()


In [ ]:
def build_keras_cnn():
    '''
    A build function for a cnn that can be used by the keras skl classifier wrapper and therefore in our experimental setup.
    '''
    kernel_size = 3
    
    inputs = keras.Input(shape=(2304,), name='image')
    # reshape to [all examples=> -1, 48*48 image, greyscale=> 1 channel]
    reshape = keras.layers.Reshape(target_shape=(48, 48, 1))(inputs)

    
    layer_1 = keras.layers.Conv2D(filters=32, kernel_size=kernel_size, activation='relu')(reshape)
    pool_1 = keras.layers.MaxPooling2D()(layer_1)

    dense_block = keras.layers.Flatten()(pool_1)
    dense_block = keras.layers.Dense(256, activation='relu')(dense_block)
    dense_block = keras.layers.Dense(256, activation='relu')(dense_block)
    fin = keras.layers.Dense(64, activation='relu')(dense_block)

    # softmax_layer = keras.layers.Dense(10, activation='softmax')(model) # ignore softmax here, use SparseCategoricalCrossentropy below

    # compile the keras model
    model = keras.Model(inputs=inputs, outputs=fin, name='cnn')

    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# Use tensorboard for visualisations
# to view, at the command prompt in this modeling directory with a local tensorflow install:
#   tensorboard --logdir tensorboard_logs/fit --host 0.0.0.0
# alternatively, see the % tensorboard cell below

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir='tensorboard_logs', histogram_freq=1, write_graph=True, write_images=True
)

cnn_clf = KerasClassifier(build_fn=build_keras_cnn, epochs=10, batch_size=100, callbacks=[tensorboard_callback])

In [ ]:
print("CNN model, original data split:")
print(f"{train_set['target'].shape[0]} training examples")
run_experiment(cnn_clf, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")

The junk architecture that I threw together as a test is slow but matches performance with the fully connected multi layer perceptron above.
Given that reduced training time is supposed to be an advantage of CNNs, this leaves some room for improvement.

# Visualize

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir tensorboard_logs --host 0.0.0.0


In [ ]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances


In [ ]:
# https://keras.io/visualization/
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils import model_to_dot
def visualize(mlp2): 
    # mlp2 = MLPClassifier(hidden_layer_sizes=(50,), max_iter=10, alpha=1e-4,
    #                     solver='sgd', verbose=10, tol=1e-4, random_state=1,
    #                     learning_rate_init=.1)

#     mlp2 = get_mlp()
#     run_experiment(mlp2, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])

    mlp2.fit(train_set.iloc[:,:-1], train_set['target'])
    print("Training set score: %f" % mlp2.score(train_set.iloc[:,:-1], train_set['target']))
    print("Test set score: %f" % mlp2.score(test_set.iloc[:,:-1], test_set['target']))

    fig, axes = plt.subplots(4, 4)
    # use global min / max to ensure all weights are shown on the same scale
    vmin, vmax = mlp2.coefs_[0].min(), mlp2.coefs_[0].max()
    for coef, ax in zip(mlp2.coefs_[0].T, axes.ravel()):
        ax.matshow(coef.reshape(48, 48), cmap=plt.cm.gray, vmin=.5 * vmin,
                   vmax=.5 * vmax)
        ax.set_xticks(())
        ax.set_yticks(())

    plt.show()

# From here on out, will only use full data set since that gave the highest accuracy previously

## Point of comparison (same code from earlier): 

In [ ]:
print("Original split:")
print(f"{train_set['target'].shape[0]} training examples")
mlp_max = get_mlp()
run_experiment(mlp_max, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")
# visualize(mlp_max)

# Adjusting learning rate:

## Adjusting the type of learning yielded no difference in accuracy value (always 98%)

In [ ]:
# Learning rate can be constant, invscaling, adaptive. First and last are straight forward, but:
#Invscaling- constant except for when loss has NOT gone down for 2 epochs

######### Default init of 0.001 with different methods ########
# print("Learning Rate Constant: ")
# mlp = MLPClassifier(random_state=42, learning_rate='constant')
# run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
# print("------\n")

# print("Learning Rate Inverse Scaling: ")
# mlp = MLPClassifier(random_state=42, learning_rate='invscaling')
# run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
# print("------\n")

# print("Learning Rate Adaptive: ")
# mlp = MLPClassifier(random_state=42, learning_rate='adaptive')
# run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
# print("------\n")

In [ ]:
######## Different learning rate init values ########

#Increase learning rate to 0.5 (learning_rate_init)
# print("Learning Rate Init = 0.5: ")
# mlp = MLPClassifier(random_state=42, learning_rate_init=0.5)
# run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
# print("------\n")

#Decrease learning rate to 0.0001 (learning_rate_init)
print("Learning Rate Init = 0.0001: ")
mlp = MLPClassifier(random_state=42, learning_rate_init=0.0001)
run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")

# Adjusting Epochs:

In [ ]:
# Epochs = max_iters in scikit

#Increase num epochs
print("Epochs = 1000")
mlp = MLPClassifier(random_state=42, max_iter=1000)
run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")
# visualize(mlp)

#Decrease num epochs
print("Epochs = 5: ")
mlp = MLPClassifier(random_state=42, max_iter=5)
run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")

# Adjusting Momentum:

In [ ]:
#Between 0 and 1, default = 0.9

print("Momentum = 1.0: ")
mlp = MLPClassifier(random_state=42, momentum=1.0)
run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")

print("Momentum = 0.5: ")
mlp = MLPClassifier(random_state=42, momentum=0.5)
run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")

print("Momentum = 0.1: ")
mlp = MLPClassifier(random_state=42, momentum=0.1)
run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")

# Adjusting Validation Threshold:

In [ ]:
#Only used if early_stopping = True (which default is false)
#Must be between 0 and 1

print("Validation Threshold = 0.1: ")
mlp = MLPClassifier(random_state=42, early_st./opping=True, validation_fraction=0.1)
run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")

print("Validation Threshold = 0.5: ")
mlp = MLPClassifier(random_state=42, early_stopping=True, validation_fraction=0.5)
run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")

print("Validation Threshold = 0.9: ")
mlp = MLPClassifier(random_state=42, early_stopping=True, validation_fraction=0.9)
run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")


# print("Validation Threshold = 0.6: ")
# mlp = MLPClassifier(random_state=42, early_stopping=True, validation_fraction=0.6)
# run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
# print("------\n")


# print("Validation Threshold = 0.75: ")
# mlp = MLPClassifier(random_state=42, early_stopping=True, validation_fraction=0.75)
# run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
# print("------\n")

# Adjusting layers/nodes/connections (via Keras):

In [ ]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# https://machinelearningmastery.com/build-multi-layer-perceptron-neural-network-models-keras/
#96% with orriginal keras model
#Baseline was 97% accuracy
def two_layer_keras():
    model = Sequential()
    model.add(Dense(1024, input_dim=2304, activation='relu'))
#     model.add(Dense(256, activation='relu'))
#     model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    print(model.summary())

    # compile the keras model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fit the keras model on the dataset
# model.fit(X_train, y_train, epochs=10, batch_size=100)

# evaluate the keras model
# loss, accuracy = model.evaluate(X_test, y_test)

kcls_2layers = KerasClassifier(build_fn=two_layer_keras, epochs=10, batch_size=100)
run_experiment(kcls_2layers, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])

#Note, if you try running this example in an IPython or Jupyter notebook you may get an error.
#The reason is the output progress bars during training. 
#You can easily turn these off by setting verbose=0 in fit() and evaluate() calls



In [ ]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# https://machinelearningmastery.com/build-multi-layer-perceptron-neural-network-models-keras/
#96% with orriginal keras model
#Baseline was 97% accuracy
def same_layer_keras():
    model = Sequential()
    model.add(Dense(1024, input_dim=2304, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    print(model.summary())

    # compile the keras model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fit the keras model on the dataset
# model.fit(X_train, y_train, epochs=10, batch_size=100)

# evaluate the keras model
# loss, accuracy = model.evaluate(X_test, y_test)

kcls_same = KerasClassifier(build_fn=same_layer_keras, epochs=10, batch_size=100)
run_experiment(kcls_same, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])

#Note, if you try running this example in an IPython or Jupyter notebook you may get an error.
#The reason is the output progress bars during training. 
#You can easily turn these off by setting verbose=0 in fit() and evaluate() calls



In [ ]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# https://machinelearningmastery.com/build-multi-layer-perceptron-neural-network-models-keras/
#96% with orriginal keras model
#Baseline was 97% accuracy
def six_layer_keras():
    model = Sequential()
    model.add(Dense(1024, input_dim=2304, activation='relu'))
    model.add(Dense(500, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    print(model.summary())

    # compile the keras model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fit the keras model on the dataset
# model.fit(X_train, y_train, epochs=10, batch_size=100)

# evaluate the keras model
# loss, accuracy = model.evaluate(X_test, y_test)

kcls_6layers = KerasClassifier(build_fn=six_layer_keras, epochs=10, batch_size=100)
run_experiment(kcls_6layers, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])

#Note, if you try running this example in an IPython or Jupyter notebook you may get an error.
#The reason is the output progress bars during training. 
#You can easily turn these off by setting verbose=0 in fit() and evaluate() calls



In [ ]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# https://machinelearningmastery.com/build-multi-layer-perceptron-neural-network-models-keras/
#96% with orriginal keras model
#Baseline was 97% accuracy
def tanh_softmax_keras():
    model = Sequential()
    model.add(Dense(1024, input_dim=2304, activation='tanh'))
#     model.add(Dense(1024, input_dim=2304, activation='softmax'))
    model.add(Dense(10, activation='softmax'))
    print(model.summary())

    # compile the keras model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

kcls_activ = KerasClassifier(build_fn=tanh_softmax_keras, epochs=10, batch_size=100)
run_experiment(kcls_activ, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])


# Maximizing all parameters:

In [ ]:
mlp = MLPClassifier(random_state=42, learning_rate_init=0.0001, max_iter=1000, early_stopping=True, validation_fraction=0.1)
run_experiment(mlp, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
print("------\n")
# visualize(mlp)

In [ ]:
#Can only work with Keras
# history = kcls.fit(train_set.iloc[:,:-1], train_set['target'], validation_split=0.25, epochs=10, batch_size=100)

# # Plot training & validation accuracy values
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(historny.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# Best performing tests visualization

In [ ]:
tensorboard_callback2 = keras.callbacks.TensorBoard(
    log_dir='tensorboard_logs_2layers', histogram_freq=1, write_graph=True, write_images=True
)

In [ ]:
# cnn_clf = KerasClassifier(build_fn=build_keras_cnn, epochs=10, batch_size=100, callbacks=[tensorboard_callback])

# mlp_validation = MLPClassifier(random_state=42, early_stopping=True, validation_fraction=0.1, callbacks=[tensorboard_callback])
kcls_2layer = KerasClassifier(build_fn=two_layer_keras, epochs=10, batch_size=100, callbacks=[tensorboard_callback2])
kcls_1024 = KerasClassifier(build_fn=same_layer_keras, epochs=10, batch_size=100, callbacks=[tensorboard_callback2])
kcls_6layer = KerasClassifier(build_fn=six_layer_keras, epochs=10, batch_size=100, callbacks=[tensorboard_callback2])

run_experiment(kcls_2layer, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
run_experiment(kcls_1024, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])
run_experiment(kcls_6layer, train_set.iloc[:,:-1], train_set['target'], test_set.iloc[:,:-1], test_set['target'])


In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir tensorboard_logs_2layers --host 0.0.0.0